<a href="https://colab.research.google.com/github/adrianacupp/fake_new_classification/blob/main/Pretrained_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
#import nltk

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd 
df =pd.read_csv('gdrive/My Drive/Colab Notebooks/df_pre_tok.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70685 entries, 0 to 70684
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   70685 non-null  int64 
 1   title        70685 non-null  object
 2   text         70685 non-null  object
 3   label        70685 non-null  int64 
 4   text_clean   70685 non-null  object
 5   title_clean  70577 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.2+ MB


In [5]:
df.dropna(axis=0, how='any', inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70577 entries, 0 to 70684
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   70577 non-null  int64 
 1   title        70577 non-null  object
 2   text         70577 non-null  object
 3   label        70577 non-null  int64 
 4   text_clean   70577 non-null  object
 5   title_clean  70577 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.8+ MB


In [8]:
import torch
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

In [9]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.2676, 0.4093, 0.2444],
        [0.6451, 0.7063, 0.7996],
        [0.4555, 0.4035, 0.7082],
        [0.6411, 0.6370, 0.4901],
        [0.5314, 0.1185, 0.1999]])


In [10]:
config = BertConfig.from_pretrained('bert-base-uncased',finetuning_task='binary')

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
def get_tokens(text, tokenizer, max_seq_length, add_special_tokens=True): 
    input_ids = tokenizer.encode(text, 
                                 add_special_tokens=add_special_tokens, 
                                 max_length=max_seq_length, 
                                 pad_to_max_length=True) 
    attention_mask = [int(id > 0) for id in input_ids] 
    assert len(input_ids) == max_seq_length 
    assert len(attention_mask) == max_seq_length 
    return (input_ids, attention_mask) 
text = "Here is the sentence I want embeddings for." 
input_ids, attention_mask = get_tokens(text, 
                                       tokenizer, 
                                       max_seq_length=30, 
                                       add_special_tokens = True) 
input_tokens = tokenizer.convert_ids_to_tokens(input_ids) 
print (text) 
print (input_tokens) 
print (input_ids) 
print (attention_mask)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Here is the sentence I want embeddings for.
['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[101, 2182, 2003, 1996, 6251, 1045, 2215, 7861, 8270, 4667, 2015, 2005, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
#XY train_test split + tokenize X_train and X_test
X_train, X_test, Y_train, Y_test = train_test_split(df['title_clean'],
                                                    df['label'], 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=df['label'])

X_train_tokens = X_train.apply(get_tokens, args=(tokenizer, 50)) 
X_test_tokens = X_test.apply(get_tokens, args=(tokenizer, 50))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [16]:
X_train_tokens.shape

(56461,)

In [17]:
Y_train.shape

(56461,)

In [18]:
#creation of 3 tensors: tokens, input masks and target labels

from torch.utils.data import TensorDataset 

input_ids_train = torch.tensor( 
    [features[0] for features in X_train_tokens.values], dtype=torch.long) 
input_mask_train = torch.tensor( 
    [features[1] for features in X_train_tokens.values], dtype=torch.long) 
label_ids_train = torch.tensor(Y_train.values, dtype=torch.long) 

print (input_ids_train.shape) 
print (input_mask_train.shape) 
print (label_ids_train.shape)

torch.Size([56461, 50])
torch.Size([56461, 50])
torch.Size([56461])


In [19]:
input_ids_train[1]

tensor([  101,  8398,  2626,  1037,  2338,  2055, 12581,  6101,  2075,  2308,
         1998, 10017, 14398,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [20]:
#combine tensors into a tensordataset

train_dataset = TensorDataset(input_ids_train,input_mask_train,label_ids_train)

In [21]:
# training the model

from torch.utils.data import DataLoader, RandomSampler

train_batch_size = 64
num_train_epochs = 2
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset,
sampler=train_sampler,
batch_size=train_batch_size)
t_total = len(train_dataloader) // num_train_epochs
print ("Num examples = ", len(train_dataset))
print ("Num Epochs = ", num_train_epochs)
print ("Total train batch size = ", train_batch_size)
print ("Total optimization steps = ", t_total)

Num examples =  56461
Num Epochs =  2
Total train batch size =  64
Total optimization steps =  441


In [22]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [23]:
learning_rate = 1e-4
adam_epsilon = 1e-8
warmup_steps = 0
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer,
num_warmup_steps=warmup_steps,
num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [24]:
from tqdm.notebook import tqdm

In [25]:
from tqdm import trange, notebook

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_iterator = trange(num_train_epochs, desc="Epoch")

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
print(device)

cuda


In [28]:
torch.cuda.is_available()

True

In [ ]:
# Put model in 'train' mode
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [29]:
for epoch in train_iterator:
    epoch_iterator = notebook.tqdm(train_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
# Reset all gradients at start of every iteration
        model.zero_grad()
# Put the model and the input observations to GPU
        model.to(device)
        batch = tuple(t.to(device) for t in batch)
# Identify the inputs to the model
        inputs = {'input_ids': batch[0],
                    'attention_mask': batch[1],
                    'labels': batch[2]}
# Forward Pass through the model. Input -> Model -> Output
        outputs = model(**inputs)
# Determine the deviation (loss)
        loss = outputs[0]
        print("\r%f" % loss, end='')
# Back-propogate the loss (automatically calculates gradients)
        loss.backward()
# Prevent exploding gradients by limiting gradients to 1.0
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
# Update the parameters and learning rate
        optimizer.step()
        scheduler.step()

Iteration:   0%|          | 0/883 [00:00<?, ?it/s]

0.184605

Epoch:  50%|█████     | 1/2 [08:19<08:19, 499.05s/it]

0.265976

Iteration:   0%|          | 0/883 [00:00<?, ?it/s]

0.139135

Epoch: 100%|██████████| 2/2 [16:10<00:00, 485.24s/it]

0.386751

In [30]:
model.save_pretrained('outputs')

In [32]:
import numpy as np
from torch.utils.data import SequentialSampler

In [33]:
test_batch_size = 64
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset,
                              sampler=test_sampler,
                              batch_size=test_batch_size)

NameError: ignored

In [ ]:
model = model.from_pretrained('/outputs')

In [ ]:
# Initialize the prediction and actual labels
preds = None
out_label_ids = None

In [ ]:
# Put model in "eval" mode
model.eval()

In [ ]:
for batch in notebook.tqdm(test_dataloader, desc="Evaluating"):
  model.to(device)
  batch = tuple(t.to(device) for t in batch)
# Do not track any gradients since in 'eval' mode
  with torch.no_grad():
      inputs = {'input_ids': batch[0],
                 'attention_mask': batch[1],
                  'labels': batch[2]}